#Setup

In [12]:
use_gdrive = False # @param {type:"boolean"}

RESULTS_PATH = "results1"

if use_gdrive:
  try:
    # mount your google drive to get permanent storage for your results
    from google.colab import drive
    drive.mount('/content/drive')

    RESULTS_PATH = "/content/drive/MyDrive/infoseclab23/results1"
  except ModuleNotFoundError:
    print("failed to mount gdrive")
else:
  print(f"saving results to '{RESULTS_PATH}'. If you're using Google Colab, this folder will be deleted when you disconnect!")

!mkdir -p {RESULTS_PATH}

saving results to 'results1'. If you're using Google Colab, this folder will be deleted when you disconnect!


Zugriff verweigert
Folgender Fehler trat auf: -p.
Zugriff verweigert
Folgender Fehler trat auf: results1.


In [7]:
import sys

# Lab files
#![ ! -d 'infoseclab_23' ] && git clone https://github.com/ethz-spylab/infoseclab_23.git
%cd infoseclab_23
!git pull https://github.com/ethz-spylab/infoseclab_23.git
%cd ..
if "infoseclab_23" not in sys.path:
  sys.path.append("infoseclab_23/infoseclab")

[WinError 2] Das System kann die angegebene Datei nicht finden: 'infoseclab_23'
c:\Users\lucas
c:\Users


fatal: not a git repository (or any of the parent directories): .git


In [6]:
###################################################################
#sketchy shit
import os
import sys
%pwd
print(os.getcwd())
sys.path.append('infoseclab_23/infoseclab')


###################################################################

c:\Users\lucas\Documents\HS23\InfSec_Lab\Part2


#Imports

In [10]:
import torch
import numpy as np
import infoseclab_23.infoseclab
from infoseclab_23.infoseclab import *
import os
from zipfile import ZipFile

device = "cuda"

ModuleNotFoundError: No module named 'infoseclab'

# 0.&nbsp; A quick primer on constrained optimization in PyTorch

To get a feel for how to optimize functions in PyTorch over a domain, below we solve a simple 1-dimensional function mimization problem.

We want to find the minimum of $f(x)$ under the constraint $x \in [-1, 1]$.

(the actual minimimum is at $x=\sqrt{2/3} \approx 0.8165$ and has value $f(x) \approx -6.089$).

In [ ]:
# the function we want to minimize
def f(x):
  return x**3 - 2*x - 5

# our starting point
x = torch.zeros(1).to(device)

for i in range(20):
  x = x.requires_grad_(True)  # we want to take gradients with respect to x

  objective = f(x)            # compute the current objective
  objective.backward()        # take the gradient of the objective with respect to all inputs
  grad = x.grad.detach()      # get the value of the gradient with respect to x

  print(f"step {i}: f({x.item():.2f}) = {objective.item():.2f}")

  with torch.no_grad():
    x = x - 0.05 * torch.sign(grad)  # take a gradient update step to minimize the objective
    x = torch.clamp(x, -1, 1)        # ensure we stay in the allowed range

step 0: f(0.00) = -5.00
step 1: f(0.05) = -5.10
step 2: f(0.10) = -5.20
step 3: f(0.15) = -5.30
step 4: f(0.20) = -5.39
step 5: f(0.25) = -5.48
step 6: f(0.30) = -5.57
step 7: f(0.35) = -5.66
step 8: f(0.40) = -5.74
step 9: f(0.45) = -5.81
step 10: f(0.50) = -5.88
step 11: f(0.55) = -5.93
step 12: f(0.60) = -5.98
step 13: f(0.65) = -6.03
step 14: f(0.70) = -6.06
step 15: f(0.75) = -6.08
step 16: f(0.80) = -6.09
step 17: f(0.85) = -6.09
step 18: f(0.80) = -6.09
step 19: f(0.85) = -6.09


# 1.&nbsp;Targeted PGD attack on undefended ResNet-50

We will first run a simple *targeted* PGD attack, where the goal is to get the model to misclassify an input `(x, y)` into a specific incorrect class `y'`.

You can design your attack however you'd like, but we recommend first
implementing a `project` method that projects an adversarial example
onto the Lp ball centered at the original sample. It is customary to
also ensure that the projected sample is in the valid range for an image (i.e.,
all pixel values in [0, 255]).



In [ ]:
class PGD(object):
    """
    A targeted PGD attack in l_inf norm.
    """
    def __init__(self, epsilon, clf):
        """
        :param epsilon: the maximum perturbation allowed
        :param clf: the classifier to attack
        """
        self.epsilon = epsilon
        self.clf = clf

    def project(self, x_adv, x_orig):
        """
        Project x_adv onto the epsilon ball around x_orig.
        :param x_adv: the adversarial images
        :param x_orig: the clean images
        :return: the adversarial images projected onto the epsilon ball, in the range [0, 255]
        """
        raise NotImplementedError()

    def attack_batch(self, x, y_targets):
        """
        Attack a batch of images with PGD.
        :param x: the batch of images (torch tensors) to attack of size (batch_size, 3, 224, 224)
        :param y_targets: the target labels of size (batch_size,)
        :return: the adversarial images of size (batch_size, 3, 224, 224)
        """
        raise NotImplementedError()

    def attack_all(self, images, labels, batch_size=20):
        """
        A utility to attack all images in the dataset by batching.
        :param images: the images to attack, of size (N, 3, 224, 224) in the range [0, 255]
        :param labels: the target labels
        :param batch_size: the batch size to use
        :return: the adversarial images
        """
        return utils.batched_func(self.attack_batch, inputs=(images, labels),
                                  batch_size=batch_size,
                                  device=self.clf.device)

# load the defense
resnet = defenses.ResNet(device)

pgd = PGD(epsilon=EPSILON, clf=resnet)
x_adv = pgd.attack_all(ImageNet.clean_images, ImageNet.targets, batch_size=6)

utils.save_images(os.path.join(RESULTS_PATH, "x_adv_targeted.npy"), x_adv)
evaluation.eval_targeted_pgd(os.path.join(RESULTS_PATH, "x_adv_targeted.npy"), device);

In [ ]:
# you can visualize your attack samples as follows if this helps
idx = 0
input = torch.stack([x_adv[idx], ImageNet.clean_images[idx]]).to(device)
logits = resnet.get_logits(input)
utils.display(x_adv[idx], image_orig=ImageNet.clean_images[idx], logits=logits)

# 2.&nbsp;Evading Detection

It turns out that "naive" adversarial examples are very easy to *detect*.
So one could build a defense that aims to detect when an input has been perturbed, to reject it and raise an alarm.

Unfortunately, as we'll see such defenses are hard to make robust against an *adaptive* attacker that also optimizes over the detector.

You will now implement attacks against two detector defenses:

<ul>
  <li> 2.1. A detector using a standard neural network. </li>
  <li> 2.2. A Random Forest detector </li>
<ul>

In [ ]:
# Your previous attack is likely easily detected
evaluation.eval_detector_attack(os.path.join(RESULTS_PATH, "x_adv_targeted.npy"), device=device);

## 2.1&nbsp;Evading A Neural Network Detector

We will first do a targeted attack against the `ResNetDetector` defense.
This defense takes the standard `ResNet` classifier from before, and adds an additional detector network.

The defense can be used for classification, in which case it outputs an array of scores for each of the 1000 classes, for each input:

```
resnet_det = ResNetDetector(device)
resnet_det.get_logits(x) -> [N, 1000]
```

To obtain a detector, we trained a *binary* classifier that takes in an input and outputs binary logits for the task of distinguishing clean images (class 0) from adversarially perturbed ones (class 1):

```
resnet_det = ResNetDetector(device)
resnet_det.get_detection_logits(x) -> [N, 2]
```

*(the classifier and detector actually share most of their implementation.
The original ResNet classifier is of the form `g(f(x))` where `f` is a <u>feature extractor</u> that maps inputs to feature vectors, and `g` is a <u>linear layer</u> that maps a feature vector to a vector of 1000 class scores.
The detector takes as input the same feature vector `f(x)`, and applies a different linear layer `g_det` that maps the features to a vector of 2 class scores.
See `infoseclab.defenses.defense_detector.ResNetDetector` for details).*

Note: You are allowed to use the `ResNetDetector` module in your attack, but you are not allowed to modify it.

In [ ]:
class PGD_Det(PGD):
    """
    A targeted PGD attack that also tries to evade detection.
    """

    def __init__(self, epsilon, clf):
        """
        :param epsilon: the maximum perturbation allowed
        :param clf: the classifier to attack
        """
        super().__init__(epsilon, clf)

    def attack_batch(self, x, y_targets):
        """
        Attack a batch of images with targeted PGD while also evading detection.
        :param x: the batch of images (torch tensors) to attack of size (batch_size, 3, 224, 224)
        :param y_targets: the target labels of size (batch_size,)
        :return: the adversarial images of size (batch_size, 3, 224, 224)
        """
        raise NotImplementedError()

defense_det = defenses.ResNetDetector(device)
pgd_det = PGD_Det(epsilon=EPSILON, clf=defense_det)
x_adv_det = pgd_det.attack_all(ImageNet.clean_images, ImageNet.targets, batch_size=6)

utils.save_images(os.path.join(RESULTS_PATH, "x_adv_detect.npy"), x_adv_det)
evaluation.eval_detector_attack(os.path.join(RESULTS_PATH, "x_adv_detect.npy"), device=device);

## 2.2&nbsp;Evading a Random Forest Detector

You will now try to attack a second detector defense.
As with the previous defense, we first classify the input in a standard way using the ResNet50 model.

But this time, the detector is an opaque "Random Forest" model that takes as
input the features from the resnet model and outputs a decision.
This is a discrete model (a Random Forest is a collection of decision trees) that cannot be easily differentiated. So you'll need
a new strategy!

**For this defense, your attack can be untargeted. That is, it suffices that the adversarial examples are classified into <i>any</i> incorrect class, as long as the detector doesn't flag the sample.**

In [ ]:
class PGD_Det_RF(PGD):
    """
    A targeted PGD attack that also tries to evade detection with a random forest.
    """

    def __init__(self, epsilon, clf):
        """
        :param epsilon: the maximum perturbation allowed
        :param clf: the classifier to attack
        """
        super().__init__(epsilon, clf)

    def attack_batch(self, x, y):
        """
        Attack a batch of images with targeted PGD while also evading detection.
        :param x: the batch of images (torch tensors) to attack of size (batch_size, 3, 224, 224)
        :param y: the labels of size (batch_size,)
        :return: the adversarial images of size (batch_size, 3, 224, 224)
        """
        raise NotImplementedError()

defense_det_rf = defenses.RFDetector(device)
pgd_det_rf = PGD_Det_RF(epsilon=EPSILON, clf=defense_det_rf)
x_adv_det_rf = pgd_det_rf.attack_all(ImageNet.clean_images, ImageNet.labels, batch_size=6)

utils.save_images(os.path.join(RESULTS_PATH, "x_adv_detect_rf.npy"), x_adv_det_rf)
evaluation.eval_rf_detector_attack(os.path.join(RESULTS_PATH, "x_adv_detect_rf.npy"), device=device);


#3.&nbsp; Preprocessing Defenses

We are now going to look at two defenses against adversarial examples that aim to resist noise by *pre-processing* the input before classifying it.

<ul>
  <li> 3.1. Blurring </li>
  <li> 3.2. Random cropping and noising </li>
  <li> 3.3. Input discretization </li>
<ul>

##3.1&nbsp; Blurring

A natural defense idea is to try and remove the noise from adversarial images. A simple way of trying to do that is to add a blurring filter.

The `ResNetBlur` defense implements this. Your goal is to create a targeted PGD attack that will defeat Blurring.
You **don't** need to ensure that the attack stays undetected.

The challenge you'll encounter is that the Blurring filter we use is not automatically differentiable. You'll likely need to find a way around that!

In [ ]:
class PGD_Blur(PGD):
    """
    A targeted PGD attack that tries to resist Blurring.
    """

    def __init__(self, epsilon, clf):
        """
        :param epsilon: the maximum perturbation allowed
        :param clf: the classifier to attack
        """
        super().__init__(epsilon, clf)


    def attack_batch(self, x, y_targets):
        """
        Attack a batch of images with targeted PGD while also resisting JPEG compression.
        :param x: the batch of images (torch tensors) to attack of size (batch_size, 3, 224, 224)
        :param y_targets: the target labels of size (batch_size,)
        :return: the adversarial images of size (batch_size, 3, 224, 224)
        """

        raise NotImplementedError()

defense_blur = defenses.ResNetBlur(device)
pgd_blur = PGD_Blur(epsilon=EPSILON, clf=defense_blur)
x_adv_blur = pgd_blur.attack_all(ImageNet.clean_images, ImageNet.targets, batch_size=6)

utils.save_images(os.path.join(RESULTS_PATH, "x_adv_blur.npy"), x_adv_blur)
evaluation.eval_blur_attack(os.path.join(RESULTS_PATH, "x_adv_blur.npy"), device);

##3.2&nbsp; Randomized cropping and noising

Another natural defense idea is to try and *randomize* the model's behavior to make it harder to create adversarial examples.

The ResNetRandom defense implements this, by randomly cropping and noising input images before classifying them.

Your goal is to create a targeted PGD attack that will defeat randomized pre-processing.
In this part, you **don't** need to ensure that the attack stays undetected.

**Note that since this defense is randomized, the evaluation results might vary slightly from one run to the next. To make sure that your attack passes our final evaluation, try to create an attack that has a few % of slack compared to the evaluation targets (e.g., if we target an adversarial accuracy below 5%, aim to ensure that your attack reaches ~3% or lower)**

In [ ]:
class PGD_Random(PGD):
    """
    A PGD attack that also tries to resist random preprocessing.
    """

    def __init__(self, epsilon, clf):
        super().__init__(epsilon, clf)

    def attack_batch(self, x, y_targets):
        """
        Attack a batch of images with targeted PGD while also evading random preprocessing.
        :param x: the batch of images (torch tensors) to attack of size (batch_size, 3, 224, 224)
        :param y_targets: the target labels of size (batch_size,)
        :return: the adversarial images of size (batch_size, 3, 224, 224)
        """
        raise NotImplementedError()


defense_random = defenses.ResNetRandom(device)
pgd_random = PGD_Random(epsilon=EPSILON, clf=defense_random)
x_adv_random = pgd_random.attack_all(ImageNet.clean_images, ImageNet.targets, batch_size=6)

utils.save_images(os.path.join(RESULTS_PATH, "x_adv_random.npy"), x_adv_random)
evaluation.eval_random_attack(os.path.join(RESULTS_PATH, "x_adv_random.npy"), device);

##3.3&nbsp; Discretized inputs

This final defense does something a bit crazy to make it hard for you to compute gradients (a lot of proposed defenses against adversarial examples used to do this, but none of them work...)

This defense discretizes the model's inputs as follows: each pixel (in the range [0, 1]) is mapped to an array of 20 binary "buckets", where the i-th bucket is set if the input pixel is greater than i/20. So for example we encode `0.0` as `[0 0 0 0 0 ... 0]`, `0.1` as `[1 1 0 0 ... 0]` and `1.0` as `[1 1 1 1 1 ... 1]`.

See the `ResNetDiscrete` defense for the implementation of this encoding.

The encoded inputs are then fed into a ResNet-style model (which was modified to take in inputs with `3*20` pixel channels instead of `3`).

Your goal is to create targeted adversarial examples that break this defense.

In [ ]:
class PGD_Discrete(PGD):
    """
    A PGD attack that also tries to resist input discretization.
    """

    def __init__(self, epsilon, clf):
        super().__init__(epsilon, clf)

    def attack_batch(self, x, y_targets):
        """
        Attack a batch of images with targeted PGD while also evading random preprocessing.
        :param x: the batch of images (torch tensors) to attack of size (batch_size, 3, 224, 224)
        :param y_targets: the target labels of size (batch_size,)
        :return: the adversarial images of size (batch_size, 3, 224, 224)
        """
        raise NotImplementedError()


defense = defenses.ResNetDiscrete(device)
pgd_discrete = PGD_Discrete(epsilon=EPSILON, clf=defense)
x_adv_discrete = pgd_discrete.attack_all(ImageNet.clean_images, ImageNet.targets, batch_size=6)

utils.save_images(os.path.join(RESULTS_PATH, "x_adv_discrete.npy"), x_adv_discrete)
evaluation.eval_discrete_attack(os.path.join(RESULTS_PATH, "x_adv_discrete.npy"), device);

# Create submission file (**upload `results1.zip` to moodle**)

In [ ]:
!zip -j -FSr "{RESULTS_PATH}/results1.zip" {RESULTS_PATH}

In [ ]:
from infoseclab.evaluation import eval_targeted_pgd, eval_detector_attack, eval_rf_detector_attack, eval_blur_attack, eval_random_attack, eval_discrete_attack
from infoseclab.submission import validate_zip1

assert validate_zip1(f"{RESULTS_PATH}/results1.zip")

with ZipFile(f"{RESULTS_PATH}/results1.zip", 'r') as zip:
    _ = eval_targeted_pgd(path=zip.open("x_adv_targeted.npy"), device=device)
    _ = eval_detector_attack(path=zip.open("x_adv_detect.npy"), device=device)
    _ = eval_rf_detector_attack(path=zip.open("x_adv_detect_rf.npy"), device=device)
    _ = eval_blur_attack(path=zip.open("x_adv_blur.npy"), device=device)
    _ = eval_random_attack(path=zip.open("x_adv_random.npy"), device=device)
    _ = eval_discrete_attack(path=zip.open("x_adv_discrete.npy"), device=device)

=== Evaluating targeted PGD ===
	clean accuracy: 100.0%
	adv accuracy: 0.0% (goal: ≤ 1.0%)
	adv target accuracy: 100.0% (goal: ≥99.0%)
SUCCESS
=== Evaluating targeted PGD with Detection ===
	clean accuracy: 100.0%
	adv accuracy: 0.0% (goal: ≤ 1.0%)
	adv target accuracy: 100.0% (goal: ≥99.0%)
	clean examples detected: 2.5% (goal: ≤5.0%)
	adv examples detected: 0.0% (goal: ≤1.0%)
SUCCESS
=== Evaluating targeted PGD on JPEG defense ===
	clean accuracy: 94.0%
	adv accuracy: 0.0% (goal: ≤ 1.0%)
	adv target accuracy: 100.0% (goal: ≥99.0%)
SUCCESS
=== Evaluating targeted PGD on random defense ===
	clean accuracy: 97.0%
	adv accuracy: 1.0% (goal: ≤ 5.0%)
	adv target accuracy: 96.5% (goal: ≥95.0%)
SUCCESS
